# Introduction to loading data <img align="right" src="../Supplementary_data/dea_logo.jpg">

* **[Sign up to the DEA Sandbox](https://app.sandbox.dea.ga.gov.au/)** to run this notebook interactively from a browser
* **Compatibility:** Notebook currently compatible with both the `NCI` and `DEA Sandbox` environments
* **Products used:** 
[ga_ls7e_nbart_gm_cyear_3](https://explorer.dea.ga.gov.au/ga_ls7e_nbart_gm_cyear_3),
[ga_ls8c_nbart_gm_cyear_3](https://explorer.dea.ga.gov.au/ga_ls8c_nbart_gm_cyear_3)
* **Prerequisites:** Users of this notebook should have a basic understanding of:
    * How to run a [Jupyter notebook](01_Jupyter_notebooks.ipynb)
    * The basic structure of the DEA [satellite datasets](02_DEA.ipynb)
    * Inspecting available [DEA products and measurements](03_Products_and_measurements.ipynb)
    

## Background
Loading data from the [Digital Earth Australia (DEA)](https://www.ga.gov.au/dea) instance of the [Open Data Cube](https://www.opendatacube.org/) requires the construction of a data query that specifies the what, where, and when of the data request.
Each query returns a [multi-dimensional xarray object](http://xarray.pydata.org/en/stable/) containing the contents of your query.
It is essential to understand the `xarray` data structures as they are fundamental to the structure of data loaded from the datacube.
Manipulations, transformations and visualisation of `xarray` objects provide datacube users with the ability to explore and analyse DEA datasets, as well as pose and answer scientific questions.

## Description
This notebook will introduce how to load data from the DEA datacube through the construction of a query and use of the `dc.load()` function.
Topics covered include:

1. Loading data using `dc.load()`
2. Interpreting the resulting `xarray.Dataset` object
    * Inspecting an individual `xarray.DataArray`
3. Customising parameters passed to the `dc.load()` function
    * Loading specific measurements
    * Loading data for coordinates in a custom coordinate reference system (CRS)
    * Projecting data to a new CRS and spatial resolution 
    * Specifying a specific spatial resampling method
4. Loading data using a reusable dictionary query
5. Loading matching data from multiple products using `like`
6. Adding a progress bar to the data load

***

## Getting started
To run this introduction to loading data from DEA, run all the cells in the notebook starting with the "Load packages" cell. For help with running notebook cells, refer back to the [Jupyter Notebooks notebook](01_Jupyter_notebooks.ipynb).

### Load packages
The `datacube` package is required to query the datacube database and load some data. 
The `with_ui_cbk` function from `odc.ui` enables a progress bar when loading large amounts of data.

In [1]:
import datacube
from odc.ui import with_ui_cbk

### Connect to the datacube
The next step is to connect to the datacube database.
The resulting `dc` datacube object can then be used to load data.
The `app` parameter is a unique name used to identify the notebook that does not have any effect on the analysis.

In [2]:
dc = datacube.Datacube(app="04_Loading_data")

## Loading data using `dc.load()`

Loading data from the datacube uses the [dc.load()](https://datacube-core.readthedocs.io/en/latest/api/indexed-data/generate/datacube.Datacube.load.html) function.

The function requires the following minimum arguments:

* `product`: The data product to load (to revise DEA products, see the [Products and measurements](03_Products_and_measurements.ipynb) notebook).
* `x`: The spatial region in the *x* dimension. By default, the *x* and *y* arguments accept queries in a geographical co-ordinate system WGS84, identified by the EPSG code *4326*.
* `y`: The spatial region in the *y* dimension. The dimensions ``longitude``/``latitude`` and ``x``/``y`` can be used interchangeably.
* `time`: The temporal extent. The time dimension can be specified using a tuple of datetime objects or strings in the "YYYY", "YYYY-MM" or "YYYY-MM-DD" format. 

For example, to load 2015 data from the [Landsat 8 NBAR-T annual geomedian product](https://explorer.dea.ga.gov.au/ga_ls8c_nbart_gm_cyear_3) for Moreton Bay in southern Queensland, use the following parameters:

* `product`: `ga_ls8c_nbart_gm_cyear_3`
* `x`: `(153.3, 153.4)`
* `y`: `(-27.5, -27.6)`
* `time`: `("2015-01-01", "2015-12-31")`

Run the following cell to load all datasets from the `ga_ls8c_nbart_gm_cyear_3` product that match this spatial and temporal extent:

In [3]:
ds = dc.load(product="ga_ls8c_nbart_gm_cyear_3",
             x=(153.3, 153.4),
             y=(-27.5, -27.6),
             time=("2015-01-01", "2015-12-31"))

ds

<xarray.Dataset>
Dimensions:      (time: 1, y: 424, x: 384)
Coordinates:
  * time         (time) datetime64[ns] 2015-07-02T11:59:59.999999
  * y            (y) float64 -3.156e+06 -3.156e+06 ... -3.168e+06 -3.168e+06
  * x            (x) float64 2.067e+06 2.067e+06 ... 2.079e+06 2.079e+06
    spatial_ref  int32 3577
Data variables:
    blue         (time, y, x) int16 469 470 474 480 481 ... 321 311 277 258 267
    green        (time, y, x) int16 510 513 517 524 527 ... 471 490 434 364 363
    red          (time, y, x) int16 231 235 238 241 242 ... 382 375 332 311 319
    nir          (time, y, x) int16 94 94 95 96 97 ... 2495 2709 2466 2136 2218
    swir1        (time, y, x) int16 56 57 56 57 58 ... 1388 1428 1181 1018 1050
    swir2        (time, y, x) int16 46 46 47 48 47 48 ... 715 710 580 491 491
    sdev         (time, y, x) float32 0.003447 0.003274 ... 0.001281 0.0019
    edev         (time, y, x) float32 132.5 127.3 125.9 ... 205.5 175.0 164.0
    bcdev        (time, y, x) float32 0.09468 0.09092 ... 0.04037 0.03832
    count        (time, y, x) int16 16 16 16 16 16 16 16 ... 13 13 12 12 12 12
Attributes:
    crs:           EPSG:3577
    grid_mapping:  spatial_ref

### Interpreting the resulting `xarray.Dataset`
The variable `ds` has returned an `xarray.Dataset` containing all data that matched the spatial and temporal query parameters inputted into `dc.load`.

*Dimensions* 

* This header identifies the number of timesteps returned in the search (`time: 1`) as well as the number of pixels in the `x` and `y` directions of the data query.

*Coordinates* 

* `time` identifies the date attributed to each returned timestep.
* `x` and `y` are the coordinates for each pixel within the spatial bounds of the query.

*Data variables*

* These are the measurements available for the nominated product. 
For every date (`time`) returned by the query, the measured value at each pixel (`y`, `x`) is returned as an array for each measurement.
Each data variable is itself an `xarray.DataArray` object ([see below](#Inspecting-an-individual-xarray.DataArray)). 

*Attributes*

* `crs` identifies the coordinate reference system (CRS) of the loaded data. 

### Inspecting an individual `xarray.DataArray`
The `xarray.Dataset` loaded above is itself a collection of individual `xarray.DataArray` objects that hold the actual data for each data variable/measurement. 
For example, all measurements listed under _Data variables_ above (e.g. `blue`, `green`, `red`, `nir`, `swir1`, `swir2`) are `xarray.DataArray` objects.

These `xarray.DataArray` objects can be inspected or interacted with by using either of the following syntaxes:
```
ds["measurement_name"]
```
or
```
ds.measurement_name
```

The ability to access individual variables means that these can be directly viewed, or further manipulated to create new variables. 
For example, run the following cell to access data from the near infra-red satellite band (i.e. `nir`):

In [4]:
ds.nir

<xarray.DataArray 'nir' (time: 1, y: 424, x: 384)>
array([[[  94,   94,   95, ...,   85,   89,   93],
        [  94,   98,   92, ...,   85,   86,  121],
        [  93,   95,   91, ...,   94,  103,  141],
        ...,
        [3155, 2848, 2651, ..., 2060, 2365, 2472],
        [2749, 2921, 2699, ..., 2081, 2270, 2477],
        [2535, 2865, 2649, ..., 2466, 2136, 2218]]], dtype=int16)
Coordinates:
  * time         (time) datetime64[ns] 2015-07-02T11:59:59.999999
  * y            (y) float64 -3.156e+06 -3.156e+06 ... -3.168e+06 -3.168e+06
  * x            (x) float64 2.067e+06 2.067e+06 ... 2.079e+06 2.079e+06
    spatial_ref  int32 3577
Attributes:
    units:         1
    nodata:        -999
    crs:           EPSG:3577
    grid_mapping:  spatial_ref

Note that the object header informs us that it is an `xarray.DataArray` containing data for the `nir` satellite band. 

Like an `xarray.Dataset`, the array also includes information about the data's **dimensions** (i.e. `(time: 1, y: 508, x: 461)`), **coordinates** and **attributes**.
This particular data variable/measurement contains some additional information that is specific to the `nir` band, including details of array's nodata value (i.e. `nodata: -999`).

> For a more in-depth introduction to `xarray` data structures, refer to the [official xarray documentation](http://xarray.pydata.org/en/stable/data-structures.html)

## Customising the `dc.load()` function

The `dc.load()` function can be tailored to refine a query.

Customisation options include:

* `measurements:` This argument is used to provide a list of measurement names to load, as listed in `dc.list_measurements()`. 
For satellite datasets, measurements contain data for each individual satellite band (e.g. near infrared). 
If not provided, all measurements for the product will be returned.
* `crs:` The coordinate reference system (CRS) of the query's `x` and `y` coordinates is assumed to be `WGS84`/`EPSG:4326` unless the `crs` field is supplied, even if the stored data is in another projection or the `output_crs` is specified. 
The `crs` parameter is required if the query's coordinates are in any other CRS.
* `group_by:` Satellite datasets based around scenes can have multiple observations per day with slightly different time stamps as the satellite collects data along its path.
These observations can be combined by reducing the `time` dimension to the day level using `group_by=solar_day`.
* `output_crs` and `resolution`: To reproject or change the resolution the data, supply the `output_crs` and `resolution` fields.    
* `resampling`: This argument allows you to specify a custom spatial resampling method to use when data is reprojected into a different CRS. 

Example syntax on the use of these options follows in the cells below.

> For help or more customisation options, run `help(dc.load)` in an empty cell or visit the function's [documentation page](https://datacube-core.readthedocs.io/en/latest/api/indexed-data/generate/datacube.Datacube.load.html)


### Specifying measurements
By default, `dc.load()` will load *all* measurements in a product.

To load data from the `red`, `green` and `blue` satellite bands only, add `measurements=["red", "green", "blue"]` to the query:

In [5]:
# Note the optional inclusion of the measurements list
ds_rgb = dc.load(product="ga_ls8c_nbart_gm_cyear_3",
                 measurements=["red", "green", "blue"],
                 x=(153.3, 153.4),
                 y=(-27.5, -27.6),
                 time=("2015-01-01", "2015-12-31"))

ds_rgb

<xarray.Dataset>
Dimensions:      (time: 1, y: 424, x: 384)
Coordinates:
  * time         (time) datetime64[ns] 2015-07-02T11:59:59.999999
  * y            (y) float64 -3.156e+06 -3.156e+06 ... -3.168e+06 -3.168e+06
  * x            (x) float64 2.067e+06 2.067e+06 ... 2.079e+06 2.079e+06
    spatial_ref  int32 3577
Data variables:
    red          (time, y, x) int16 231 235 238 241 242 ... 382 375 332 311 319
    green        (time, y, x) int16 510 513 517 524 527 ... 471 490 434 364 363
    blue         (time, y, x) int16 469 470 474 480 481 ... 321 311 277 258 267
Attributes:
    crs:           EPSG:3577
    grid_mapping:  spatial_ref

Note that the **Data variables** component of the `xarray.Dataset` now includes only the measurements specified in the query (i.e. the `red`, `green` and `blue` satellite bands).

### Loading data for coordinates in any CRS
By default, `dc.load()` assumes that the queried `x` and `y` coordinates are in the `WGS84`/`EPSG:4326` CRS.
If these coordinates are in a different coordinate system, specify this using the `crs` parameter.

The example cell below loads data for a set of `x` and `y` coordinates defined in Australian Albers (`EPSG:3577`), ensuring that the `dc.load()` function accounts for this by including `crs="EPSG:3577"`:

In [6]:
# Note the new `x` and `y` coordinates and `crs` parameter
ds_custom_crs = dc.load(product="ga_ls8c_nbart_gm_cyear_3",
                        time=("2015-01-01", "2015-12-31"),
                        x=(2069310, 2077064),
                        y=(-3155823, -3168513),
                        crs="EPSG:3577")

ds_custom_crs

<xarray.Dataset>
Dimensions:      (time: 1, y: 423, x: 259)
Coordinates:
  * time         (time) datetime64[ns] 2015-07-02T11:59:59.999999
  * y            (y) float64 -3.156e+06 -3.156e+06 ... -3.168e+06 -3.168e+06
  * x            (x) float64 2.069e+06 2.069e+06 ... 2.077e+06 2.077e+06
    spatial_ref  int32 3577
Data variables:
    blue         (time, y, x) int16 462 462 459 456 458 ... 424 444 441 399 378
    green        (time, y, x) int16 476 475 470 469 470 ... 481 491 484 450 437
    red          (time, y, x) int16 213 211 208 208 209 ... 249 251 245 222 213
    nir          (time, y, x) int16 82 83 80 78 78 79 ... 99 109 115 112 87 78
    swir1        (time, y, x) int16 50 48 48 47 46 47 46 ... 56 65 65 63 46 38
    swir2        (time, y, x) int16 41 40 39 37 36 38 37 ... 45 52 51 49 35 30
    sdev         (time, y, x) float32 0.003467 0.00352 ... 0.005748 0.005211
    edev         (time, y, x) float32 94.9 94.76 87.63 ... 183.1 109.6 116.1
    bcdev        (time, y, x) float32 0.07639 0.07612 0.06811 ... 0.08707 0.0849
    count        (time, y, x) int16 17 17 17 17 17 17 17 ... 16 17 16 16 14 14
Attributes:
    crs:           EPSG:3577
    grid_mapping:  spatial_ref

### CRS reprojection
Certain applications may require that data is output into a specific CRS.
Data can be reprojected by specifying the new `output_crs` and identifying the `resolution` required.

The example cell below reprojects data to a new CRS (UTM Zone 56S, `EPSG:32756`) and resolution (250 x 250 m). Note that for most CRSs, the first resolution value is negative (e.g. `(-250, 250)`):

In [7]:
ds_reprojected = dc.load(product="ga_ls8c_nbart_gm_cyear_3",
                         x=(153.3, 153.4),
                         y=(-27.5, -27.6),
                         time=("2015-01-01", "2015-12-31"),
                         output_crs="EPSG:32756",
                         resolution=(-250, 250))

ds_reprojected

<xarray.Dataset>
Dimensions:      (time: 1, y: 45, x: 40)
Coordinates:
  * time         (time) datetime64[ns] 2015-07-02T11:59:59.999999
  * y            (y) float64 6.958e+06 6.958e+06 ... 6.947e+06 6.947e+06
  * x            (x) float64 5.296e+05 5.299e+05 ... 5.391e+05 5.394e+05
    spatial_ref  int32 32756
Data variables:
    blue         (time, y, x) int16 456 447 440 432 424 ... 422 429 420 407 423
    green        (time, y, x) int16 464 444 429 418 401 ... 493 491 479 466 476
    red          (time, y, x) int16 207 195 187 179 170 ... 271 252 241 240 243
    nir          (time, y, x) int16 81 78 79 75 75 77 75 ... 86 86 88 86 93 107
    swir1        (time, y, x) int16 48 46 46 43 42 42 43 ... 43 45 47 45 51 65
    swir2        (time, y, x) int16 40 37 38 35 34 35 35 ... 35 37 38 36 42 54
    sdev         (time, y, x) float32 0.003807 0.003953 ... 0.005075 0.01121
    edev         (time, y, x) float32 98.53 98.11 96.56 ... 117.3 124.9 166.8
    bcdev        (time, y, x) float32 0.07894 0.07555 0.07484 ... 0.1178 0.1544
    count        (time, y, x) int16 17 17 17 17 16 16 16 ... 14 15 15 15 15 15
Attributes:
    crs:           EPSG:32756
    grid_mapping:  spatial_ref

Note that the `crs` attribute in the **Attributes** section has changed to `EPSG:32756`. 
Due to the larger 250 m resolution, there are now fewer pixels on the `x` and `y` dimensions (e.g. `x: 40, y: 45` compared to `x: 461, y: 508` in earlier examples).


### Spatial resampling methods
When a product is re-projected to a different CRS and/or resolution, the new pixel grid may differ from the original input pixels by size, number and alignment.
It is therefore necessary to apply a spatial "resampling" rule that allocates input pixel values into the new pixel grid.

By default, `dc.load()` resamples pixel values using "nearest neighbour" resampling, which allocates each new pixel with the value of the closest input pixel.
Depending on the type of data and the analysis being run, this may not be the most appropriate choice (e.g. for continuous data).

The `resampling` parameter in `dc.load()` allows you to choose a custom resampling method from the following options: 

```
"nearest", "cubic", "bilinear", "cubic_spline", "lanczos", 
"average", "mode", "gauss", "max", "min", "med", "q1", "q3"
```

The example cell below requests that all loaded data is resampled using "average" resampling:

In [8]:
# Note the additional `resampling` parameter
ds_averesampling = dc.load(product="ga_ls8c_nbart_gm_cyear_3",
                               x=(153.3, 153.4),
                               y=(-27.5, -27.6),
                               time=("2015-01-01", "2015-12-31"),
                               resolution=(-250, 250),
                               resampling="average")

ds_averesampling


<xarray.Dataset>
Dimensions:      (time: 1, y: 51, x: 47)
Coordinates:
  * time         (time) datetime64[ns] 2015-07-02T11:59:59.999999
  * y            (y) float64 -3.156e+06 -3.156e+06 ... -3.168e+06 -3.168e+06
  * x            (x) float64 2.067e+06 2.068e+06 ... 2.079e+06 2.079e+06
    spatial_ref  int32 3577
Data variables:
    blue         (time, y, x) int16 483 473 478 481 492 ... 471 573 371 321 285
    green        (time, y, x) int16 524 516 528 534 547 ... 567 704 443 490 501
    red          (time, y, x) int16 240 235 245 256 264 ... 446 616 261 412 367
    nir          (time, y, x) int16 100 91 87 88 90 87 ... 284 412 194 1869 2896
    swir1        (time, y, x) int16 60 53 49 51 53 52 ... 46 130 203 96 926 1260
    swir2        (time, y, x) int16 49 44 40 41 42 42 ... 35 93 131 59 464 584
    sdev         (time, y, x) float32 0.003118 0.00266 ... 0.00298 0.0007673
    edev         (time, y, x) float32 129.6 118.5 118.7 ... 160.0 181.4 189.9
    bcdev        (time, y, x) float32 0.09975 0.09054 0.0885 ... 0.05167 0.03022
    count        (time, y, x) int16 17 17 16 15 16 16 17 ... 14 14 14 13 12 12
Attributes:
    crs:           EPSG:3577
    grid_mapping:  spatial_ref

Python dictionaries can be used to request different sampling methods for different measurements. 
This can be particularly useful when some measurements contain contain categorical data which require resampling methods such as "nearest" or "mode" that do not modify the input pixel values.

The example cell below specifies `resampling={"red": "nearest", "*": "average"}`, which implements "nearest" neighbour resampling for the `red` satellite band only. `"*": "average"` will apply "average" resampling for all other satellite bands:


In [9]:
ds_customresampling = dc.load(product="ga_ls8c_nbart_gm_cyear_3",
                              x=(153.3, 153.4),
                              y=(-27.5, -27.6),
                              time=("2015-01-01", "2015-12-31"),
                              output_crs="EPSG:32756",
                              resolution=(-250, 250),
                              resampling={"red": "nearest", "*": "average"})

ds_customresampling

<xarray.Dataset>
Dimensions:      (time: 1, y: 45, x: 40)
Coordinates:
  * time         (time) datetime64[ns] 2015-07-02T11:59:59.999999
  * y            (y) float64 6.958e+06 6.958e+06 ... 6.947e+06 6.947e+06
  * x            (x) float64 5.296e+05 5.299e+05 ... 5.391e+05 5.394e+05
    spatial_ref  int32 32756
Data variables:
    blue         (time, y, x) int16 458 450 439 433 424 ... 426 424 420 408 416
    green        (time, y, x) int16 469 451 428 419 402 ... 505 487 480 467 471
    red          (time, y, x) int16 207 195 187 179 170 ... 271 252 241 240 243
    nir          (time, y, x) int16 81 79 78 76 75 77 75 ... 84 91 88 86 91 102
    swir1        (time, y, x) int16 48 46 46 44 42 42 43 ... 42 47 46 45 49 60
    swir2        (time, y, x) int16 40 38 38 36 34 35 35 ... 34 38 38 36 40 49
    sdev         (time, y, x) float32 0.003786 0.003991 ... 0.004875 0.0102
    edev         (time, y, x) float32 99.68 99.1 99.31 ... 116.5 124.3 160.1
    bcdev        (time, y, x) float32 0.07825 0.07446 0.07933 ... 0.1138 0.1466
    count        (time, y, x) int16 17 17 17 17 16 16 16 ... 14 15 15 15 15 15
Attributes:
    crs:           EPSG:32756
    grid_mapping:  spatial_ref

> For more information about spatial resampling methods, see the [following guide](https://rasterio.readthedocs.io/en/stable/topics/resampling.html)

## Loading data using the query dictionary syntax
It is often useful to re-use a set of query parameters to load data from multiple products.
To achieve this, load data using the "query dictionary" syntax.
This involves placing the query parameters inside a Python dictionary object which can be re-used for multiple data loads:

In [10]:
query = {"x": (153.3, 153.4),
         "y": (-27.5, -27.6),
         "time": ("2015-01-01", "2015-12-31")}


The query dictionary object can be added as an input to `dc.load()`. 

> The `**` syntax below is Python's "keyword argument unpacking" operator.
This operator takes the named query parameters listed in the query dictionary (e.g. `"x": (153.3, 153.4)`), and "unpacks" them into the `dc.load()` function as new arguments. 
For more information about unpacking operators, refer to the [Python documentation](https://docs.python.org/3/tutorial/controlflow.html#unpacking-argument-lists)

In [11]:
ds = dc.load(product="ga_ls8c_nbart_gm_cyear_3",
             **query)

ds

<xarray.Dataset>
Dimensions:      (time: 1, y: 424, x: 384)
Coordinates:
  * time         (time) datetime64[ns] 2015-07-02T11:59:59.999999
  * y            (y) float64 -3.156e+06 -3.156e+06 ... -3.168e+06 -3.168e+06
  * x            (x) float64 2.067e+06 2.067e+06 ... 2.079e+06 2.079e+06
    spatial_ref  int32 3577
Data variables:
    blue         (time, y, x) int16 469 470 474 480 481 ... 321 311 277 258 267
    green        (time, y, x) int16 510 513 517 524 527 ... 471 490 434 364 363
    red          (time, y, x) int16 231 235 238 241 242 ... 382 375 332 311 319
    nir          (time, y, x) int16 94 94 95 96 97 ... 2495 2709 2466 2136 2218
    swir1        (time, y, x) int16 56 57 56 57 58 ... 1388 1428 1181 1018 1050
    swir2        (time, y, x) int16 46 46 47 48 47 48 ... 715 710 580 491 491
    sdev         (time, y, x) float32 0.003447 0.003274 ... 0.001281 0.0019
    edev         (time, y, x) float32 132.5 127.3 125.9 ... 205.5 175.0 164.0
    bcdev        (time, y, x) float32 0.09468 0.09092 ... 0.04037 0.03832
    count        (time, y, x) int16 16 16 16 16 16 16 16 ... 13 13 12 12 12 12
Attributes:
    crs:           EPSG:3577
    grid_mapping:  spatial_ref

Query dictionaries can contain any set of parameters that would usually be provided to `dc.load()`:

In [12]:
query = {"x": (153.3, 153.4),
         "y": (-27.5, -27.6),
         "time": ("2015-01-01", "2015-12-31"),
         "output_crs": "EPSG:32756",
         "resolution": (-250, 250)}

ds_ls8 = dc.load(product="ga_ls8c_nbart_gm_cyear_3",
                 **query)

ds_ls8


<xarray.Dataset>
Dimensions:      (time: 1, y: 45, x: 40)
Coordinates:
  * time         (time) datetime64[ns] 2015-07-02T11:59:59.999999
  * y            (y) float64 6.958e+06 6.958e+06 ... 6.947e+06 6.947e+06
  * x            (x) float64 5.296e+05 5.299e+05 ... 5.391e+05 5.394e+05
    spatial_ref  int32 32756
Data variables:
    blue         (time, y, x) int16 456 447 440 432 424 ... 422 429 420 407 423
    green        (time, y, x) int16 464 444 429 418 401 ... 493 491 479 466 476
    red          (time, y, x) int16 207 195 187 179 170 ... 271 252 241 240 243
    nir          (time, y, x) int16 81 78 79 75 75 77 75 ... 86 86 88 86 93 107
    swir1        (time, y, x) int16 48 46 46 43 42 42 43 ... 43 45 47 45 51 65
    swir2        (time, y, x) int16 40 37 38 35 34 35 35 ... 35 37 38 36 42 54
    sdev         (time, y, x) float32 0.003807 0.003953 ... 0.005075 0.01121
    edev         (time, y, x) float32 98.53 98.11 96.56 ... 117.3 124.9 166.8
    bcdev        (time, y, x) float32 0.07894 0.07555 0.07484 ... 0.1178 0.1544
    count        (time, y, x) int16 17 17 17 17 16 16 16 ... 14 15 15 15 15 15
Attributes:
    crs:           EPSG:32756
    grid_mapping:  spatial_ref

After specifying the reusable query, it can be easily used to load data from a different product.
The example cell below loads Landsat 7 data for the same extent, time, output CRS and resolution as the previously loaded Landsat 8 data:

In [13]:
ds_ls7 = dc.load(product="ga_ls7e_nbart_gm_cyear_3",
                 **query)

ds_ls7

<xarray.Dataset>
Dimensions:      (time: 1, y: 45, x: 40)
Coordinates:
  * time         (time) datetime64[ns] 2015-07-02T11:59:59.999999
  * y            (y) float64 6.958e+06 6.958e+06 ... 6.947e+06 6.947e+06
  * x            (x) float64 5.296e+05 5.299e+05 ... 5.391e+05 5.394e+05
    spatial_ref  int32 32756
Data variables:
    blue         (time, y, x) int16 450 425 388 385 394 ... 408 382 367 364 383
    green        (time, y, x) int16 488 454 414 406 408 ... 510 480 467 466 465
    red          (time, y, x) int16 254 233 206 203 205 ... 329 292 272 277 273
    nir          (time, y, x) int16 157 150 134 141 142 ... 160 144 137 150 165
    swir1        (time, y, x) int16 84 85 77 79 84 80 82 ... 111 82 80 72 81 97
    swir2        (time, y, x) int16 71 71 64 71 70 66 68 ... 97 73 65 60 65 80
    sdev         (time, y, x) float32 0.007338 0.009259 ... 0.01109 0.009572
    edev         (time, y, x) float32 267.4 232.7 172.5 ... 202.2 200.9 227.4
    bcdev        (time, y, x) float32 0.2071 0.2088 0.1745 ... 0.1806 0.2081
    count        (time, y, x) int16 13 13 12 12 12 12 12 ... 13 11 11 11 12 12
Attributes:
    crs:           EPSG:32756
    grid_mapping:  spatial_ref

## Other helpful tricks
### Loading data "like" another dataset
Another option for loading matching data from multiple products is to use `dc.load()`'s `like` parameter.
This will copy the spatial and temporal extent and the CRS/resolution from an existing dataset, and use these parameters to load new data from a new product.

The example cell below loads another Landsat 7 dataset that exactly matches the `ds_ls8` dataset loaded earlier:


In [14]:
ds_ls7 = dc.load(product="ga_ls7e_nbart_gm_cyear_3",
                 like=ds_ls8)

ds_ls7

<xarray.Dataset>
Dimensions:      (time: 1, y: 45, x: 40)
Coordinates:
  * time         (time) datetime64[ns] 2015-07-02T11:59:59.999999
  * y            (y) float64 6.958e+06 6.958e+06 ... 6.947e+06 6.947e+06
  * x            (x) float64 5.296e+05 5.299e+05 ... 5.391e+05 5.394e+05
    spatial_ref  int32 32756
Data variables:
    blue         (time, y, x) int16 450 425 388 385 394 ... 408 382 367 364 383
    green        (time, y, x) int16 488 454 414 406 408 ... 510 480 467 466 465
    red          (time, y, x) int16 254 233 206 203 205 ... 329 292 272 277 273
    nir          (time, y, x) int16 157 150 134 141 142 ... 160 144 137 150 165
    swir1        (time, y, x) int16 84 85 77 79 84 80 82 ... 111 82 80 72 81 97
    swir2        (time, y, x) int16 71 71 64 71 70 66 68 ... 97 73 65 60 65 80
    sdev         (time, y, x) float32 0.007338 0.009259 ... 0.01109 0.009572
    edev         (time, y, x) float32 267.4 232.7 172.5 ... 202.2 200.9 227.4
    bcdev        (time, y, x) float32 0.2071 0.2088 0.1745 ... 0.1806 0.2081
    count        (time, y, x) int16 13 13 12 12 12 12 12 ... 13 11 11 11 12 12
Attributes:
    crs:           PROJCS["WGS 84 / UTM zone 56S",GEOGCS["WGS 84",DATUM["WGS_...
    grid_mapping:  spatial_ref

### Adding a progress bar
When loading large amounts of data, it can be useful to view the progress of the data load. 
The `progress_cbk` parameter in `dc.load()` adds a progress bar that indicates how the load is progressing:

![Progress bar](../Supplementary_data/04_Loading_data/progress_bar.jpg)

The example cell below loads 5 years of data (2013, 2014, 2015, 2016 and 2017) from the `ls8_nbart_geomedian_annual` product with a progress bar:

In [15]:
query = {"x": (153.3, 153.4),
         "y": (-27.5, -27.6),
         "time": ("2013", "2017")}

ds_progress = dc.load(product="ga_ls8c_nbart_gm_cyear_3",
                      progress_cbk=with_ui_cbk(),
                      **query)

ds_progress

<xarray.Dataset>
Dimensions:      (time: 5, y: 424, x: 384)
Coordinates:
  * time         (time) datetime64[ns] 2013-07-02T11:59:59.999999 ... 2017-07...
  * y            (y) float64 -3.156e+06 -3.156e+06 ... -3.168e+06 -3.168e+06
  * x            (x) float64 2.067e+06 2.067e+06 ... 2.079e+06 2.079e+06
    spatial_ref  int32 3577
Data variables:
    blue         (time, y, x) int16 490 506 504 497 498 ... 350 351 311 296 292
    green        (time, y, x) int16 537 557 558 548 536 ... 526 531 450 392 391
    red          (time, y, x) int16 253 273 275 269 268 ... 407 402 351 337 340
    nir          (time, y, x) int16 93 106 104 100 108 ... 2889 2557 2344 2403
    swir1        (time, y, x) int16 51 61 62 58 68 ... 1394 1397 1127 1009 1089
    swir2        (time, y, x) int16 41 48 48 47 55 49 ... 661 650 521 465 495
    sdev         (time, y, x) float32 0.003682 0.004547 ... 0.001669 0.002302
    edev         (time, y, x) float32 142.1 160.1 159.9 ... 220.8 205.4 237.7
    bcdev        (time, y, x) float32 0.1025 0.1248 0.1216 ... 0.04438 0.05313
    count        (time, y, x) int16 10 11 11 11 12 11 11 ... 17 17 15 14 14 14
Attributes:
    crs:           EPSG:3577
    grid_mapping:  spatial_ref

## Recommended next steps

To continue working through the notebooks in this beginner's guide, the following notebooks are designed to be worked through in the following order:

1. [Jupyter Notebooks](01_Jupyter_notebooks.ipynb)
2. [Digital Earth Australia](02_DEA.ipynb)
3. [Products and measurements](03_Products_and_measurements.ipynb)
4. **Loading data (this notebook)**
5. [Plotting](05_Plotting.ipynb)
6. [Performing a basic analysis](06_Basic_analysis.ipynb)
7. [Introduction to Numpy](07_Intro_to_numpy.ipynb)
8. [Introduction to Xarray](08_Intro_to_xarray.ipynb)
9. [Parallel processing with Dask](09_Parallel_processing_with_Dask.ipynb)

Once you have worked through the beginner's guide, you can join advanced users by exploring:

* A demonstration of how to load cloud-free observations in the [using load_ard](../How_to_guides/Using_load_ard.ipynb) notebook.
* The "DEA products" directory in the repository, where you can explore DEA products in depth.
* The "How_to_guides" directory, which contains a recipe book of common techniques and methods for analysing DEA data.
* The "Real_world_examples" directory, which provides more complex workflows and analysis case studies.

***
## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/GeoscienceAustralia/dea-notebooks).

**Last modified:** December 2023

**Compatible datacube version:** 

In [16]:
print(datacube.__version__)

1.8.6


## Tags
<!-- Browse all available tags on the DEA User Guide's [Tags Index](https://knowledge.dea.ga.gov.au/genindex/) -->